In [1]:
# your code here
import pandas as pd
import json

def init_ds(json):
    ds= {}
    keys = json.keys()
    for k in keys:
        ds[k]= []
    return ds, keys

def read_json(file):
    dataset = {}
    keys = []
    with open(file) as file_lines:
        for count, line in enumerate(file_lines):
            data = json.loads(line.strip())
            if count ==0:
                dataset, keys = init_ds(data)
            for k in keys:
                dataset[k].append(data[k])
                
        return pd.DataFrame(dataset)
    

reviews = read_json("yelp_dataset/yelp_academic_dataset_review.json")

businesses = read_json("yelp_dataset/yelp_academic_dataset_business.json")

businesses = businesses.dropna(axis = 0,subset = ["categories"])
businesses =  pd.concat([businesses.drop("attributes",axis = 1),businesses["attributes"].apply(pd.Series)], axis = 1)

restaurants = businesses.loc[businesses["categories"].str.contains("Restaurants")]

merged = pd.merge(reviews,restaurants, how = "inner",on = "business_id")

FileNotFoundError: [Errno 2] No such file or directory: 'yelp_dataset/yelp_academic_dataset_review.json'

In [2]:
from database import Database

db = Database("NLP_project.db")



In [3]:
merged = merged.dropna(axis = 0, subset = ["RestaurantsPriceRange2"])
price_ranges = merged.groupby("RestaurantsPriceRange2").agg({"RestaurantsPriceRange2": "count"})



In [4]:
merged = merged.loc[(merged["RestaurantsPriceRange2"] == "3") | (merged["RestaurantsPriceRange2"] == "4")]
length = merged["text"].str.split().str.len()
merged = merged.loc[(length >= 5) & (length <= 200)]


In [5]:
merged = merged.rename({"stars_x":"stars"},axis=1)
cols =  list(reviews.columns) + ["name","city","state","stars_y","categories","RestaurantsPriceRange2"]
merged = merged[cols]


In [6]:
merged = merged.drop_duplicates()

In [26]:
from sklearn.model_selection import train_test_split
#remainder, train_lm = train_test_split(merged, test_split = 0.142857)
high_rating = merged.loc[merged["stars"] >3]
remainder, high_used = train_test_split(high_rating, test_size = 0.4)


In [36]:
complete_for_use = pd.concat([merged.loc[merged["stars"] <= 3],high_used],axis = 0).reset_index(drop = True)


In [37]:
db.insert_Into("All_High_End_Balanced",complete_for_use)